In [42]:
# EDA e Visualização de dados
import pandas as pd
import plotly.express as px
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Machine Learning
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, pairwise_distances
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

# Otimização Hiperparâmetros
import optuna

### Carregar os dados

In [43]:
# Carga de dados
df_clientes = pd.read_csv('./dataset/dataset_clientes_pj.csv')

In [44]:
# Visualizar estrutura
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   atividade_economica     500 non-null    object 
 1   faturamento_mensal      500 non-null    float64
 2   numero_de_funcionarios  500 non-null    int64  
 3   localizacao             500 non-null    object 
 4   idade                   500 non-null    int64  
 5   inovacao                500 non-null    int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 23.6+ KB


In [45]:
df_clientes.head(10)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao
0,Comércio,713109.95,12,Rio de Janeiro,6,1
1,Comércio,790714.38,9,São Paulo,15,0
2,Comércio,1197239.33,17,São Paulo,4,9
3,Indústria,449185.78,15,São Paulo,6,0
4,Agronegócio,1006373.16,15,São Paulo,15,8
5,Serviços,1629562.41,16,Rio de Janeiro,11,4
6,Serviços,771179.95,13,Vitória,0,1
7,Serviços,707837.61,16,São Paulo,10,6
8,Comércio,888983.66,17,Belo Horizonte,10,1
9,Indústria,1098512.64,13,Rio de Janeiro,9,3


### EDA

In [46]:
# Distribuição da variável inovação
percentual_inovacao = df_clientes.value_counts('inovacao') / len(df_clientes) * 100
px.bar(percentual_inovacao, color=percentual_inovacao.index)

In [47]:
# Teste ANOVA (análise de variância)
# Verifica se há variações significativas na média de faturamento mensal para diferentes níveis de inovação
# Suposições / Pressupostos:
# -Observações independentes
# -Variável dependente é contínua
# -Segue uma distribuição normal
# -Homogeneidade das variâncias
# -Amostras sejam de tamanhos iguais

In [48]:
# Checar se as variâncias (faturamento) entre os grupos (inovação) são homogêneas
# Teste Barlett
# H0 - Variâncias são iguais
# H1 - Variâncias não são iguais

from scipy.stats import bartlett

# Separando os dados de faturamento em grupos com base na coluna 'inovacao'
dados_agrupados = [df_clientes['faturamento_mensal'][df_clientes['inovacao'] ==grupo] for grupo in df_clientes['inovacao'].unique()]

# Executar teste de Bartlett
bartlett_test_statistic, bartlett_p_value = bartlett(*dados_agrupados)

# Exibir resultados
print(f'Estatistica do teste de Bartlett: {bartlett_test_statistic}')
print(f'P-Value do teste de Bartlett: {bartlett_p_value}')

Estatistica do teste de Bartlett: 10.901203117231173
P-Value do teste de Bartlett: 0.28254182954905804


In [49]:
# Teste de normalidade - Shapiro-Wilk
# Verificar normalidade de distribuição dos dados
# H0 - Segue distribuição normal
# H1 - Não segue distribuição normal

# Executar o teste
from scipy.stats import shapiro
shapiro_test_statistic, shapiro_p_value = shapiro(df_clientes['faturamento_mensal'])

# Exibir resultados
print(f'Estatistica do teste de Shapiro-Wilk: {shapiro_test_statistic}')
print(f'P-Value do teste de Shapiro-Wilk: {shapiro_p_value}')

Estatistica do teste de Shapiro-Wilk: 0.9959857602472711
P-Value do teste de Shapiro-Wilk: 0.23513451034389005


In [50]:
# Aplicar a ANOVA de Welch, pois as amostras são de tamanhos diferentes
# H0 - Não há diferenças significativas entre as médias dos grupos
# H1 - Há pelo menos uma diferença significativa entre as médias dos grupos
from pingouin import welch_anova

aov = welch_anova(dv= 'faturamento_mensal', between= 'inovacao', data=df_clientes)

# Exibir resultados
print(f'Estatistica do teste de ANOVA Welch: {aov.loc[0, "F"]}')
print(f'P-Value do teste de ANOVA Welch: {aov.loc[0, "p-unc"]}')

Estatistica do teste de ANOVA Welch: 1.1269836194061693
P-Value do teste de ANOVA Welch: 0.34526211273911467


### Treinar o algoritmo K-Means

In [51]:
# Selecionar as colunas para clusterização
X = df_clientes.copy()

# Separação entre variáveis numéricas, categóricas e categóricas ordinais
numeric_features = ['faturamento_mensal', 'numero_de_funcionarios', 'idade']
categorical_features = ['localizacao', 'atividade_economica']
ordinal_features = ['inovacao']

# Aplicar transformações por tipo de variável
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()
ordinal_transformer = OrdinalEncoder()

# Criar o preprocessador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('ord', ordinal_transformer, ordinal_features)
    ]
)

# Transformar os dados
X_transformed = preprocessor.fit_transform(X)

In [52]:
X_transformed

array([[-0.74634498, -0.54179191, -1.10058849, ...,  0.        ,
         0.        ,  1.        ],
       [-0.56165548, -1.5035527 ,  1.94344851, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.40582654,  1.06114274, -1.77704115, ...,  0.        ,
         0.        ,  9.        ],
       ...,
       [ 2.8196246 , -1.18296577,  0.25231684, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.03321411, -0.54179191, -1.43881482, ...,  0.        ,
         0.        ,  3.        ],
       [-2.03011486, -0.22120498, -1.77704115, ...,  1.        ,
         0.        ,  9.        ]])

In [53]:
# Optuna para otimização de hiperparâmetros
def kmeans_objective(trial):
    # Definir hiperparâmetros a serem ajustados
    n_clusters = trial.suggest_int('n_clusters', 3, 10)
    distance_metric = trial.suggest_categorical('distance_metric', ['euclidean', 'minkowski'])
    
    # Criar o modelo KMeans
    modelo_kmeans = KMeans(n_clusters=n_clusters, random_state=51)
    
    # Treinar o modelo
    modelo_kmeans.fit(X_transformed)
    
    # Calcular o Silhouette Score
    distances = pairwise_distances(X_transformed, metric=distance_metric)
    silhouette_avg = silhouette_score(distances, modelo_kmeans.labels_)
    return silhouette_avg

In [55]:
# Criar um estudo Optuna

search_space = {'n_clusters': [3, 4, 5, 6, 7, 8, 9, 10], 'distance_metric': ['euclidean', 'minkowski']}
sampler = optuna.samplers.GridSampler(search_space=search_space)
estudo_kmeans = optuna.create_study(direction='maximize', sampler=sampler)

# Rodar o estudo
estudo_kmeans.optimize(kmeans_objective, n_trials=100)

[I 2025-05-06 12:57:08,255] A new study created in memory with name: no-name-94b51383-3d18-4717-97db-ddeae90e44ee
[I 2025-05-06 12:57:08,278] Trial 0 finished with value: 0.3847879496502923 and parameters: {'n_clusters': 4, 'distance_metric': 'euclidean'}. Best is trial 0 with value: 0.3847879496502923.
[I 2025-05-06 12:57:08,297] Trial 1 finished with value: 0.1123004838158908 and parameters: {'n_clusters': 9, 'distance_metric': 'euclidean'}. Best is trial 0 with value: 0.3847879496502923.
[I 2025-05-06 12:57:08,312] Trial 2 finished with value: 0.44454582909990875 and parameters: {'n_clusters': 3, 'distance_metric': 'minkowski'}. Best is trial 2 with value: 0.44454582909990875.
[I 2025-05-06 12:57:08,325] Trial 3 finished with value: 0.3847879496502923 and parameters: {'n_clusters': 4, 'distance_metric': 'minkowski'}. Best is trial 2 with value: 0.44454582909990875.
[I 2025-05-06 12:57:08,343] Trial 4 finished with value: 0.14731572416665714 and parameters: {'n_clusters': 8, 'distanc

In [58]:
# Melhor configuração encontrada pelo Optuna
best_params = estudo_kmeans.best_params

# Instanciando o modelo KMeans com os melhores parâmetros
best_kmeans = KMeans(n_clusters=best_params['n_clusters'], random_state=51)
best_kmeans.fit(X_transformed)

# Calcular o Silhouette Score
distances = pairwise_distances(X_transformed, metric=best_params['distance_metric'])
best_silhouette = silhouette_score(distances, best_kmeans.labels_)

print(f'k (Numero de Clusters): {best_params["n_clusters"]}')
print(f'Métrica de Distância Selecionada: {best_params["distance_metric"]}')
print(f'Silhouette Score: {best_silhouette}')

k (Numero de Clusters): 3
Métrica de Distância Selecionada: euclidean
Silhouette Score: 0.4445458290999088


In [59]:
# Criar coluna com cluster escolhido
df_clientes['cluster'] = best_kmeans.labels_


In [60]:
# Visualizar os primeiros registros com os cluters inclusos
df_clientes.head(10)

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao,cluster
0,Comércio,713109.95,12,Rio de Janeiro,6,1,0
1,Comércio,790714.38,9,São Paulo,15,0,0
2,Comércio,1197239.33,17,São Paulo,4,9,1
3,Indústria,449185.78,15,São Paulo,6,0,0
4,Agronegócio,1006373.16,15,São Paulo,15,8,1
5,Serviços,1629562.41,16,Rio de Janeiro,11,4,2
6,Serviços,771179.95,13,Vitória,0,1,0
7,Serviços,707837.61,16,São Paulo,10,6,1
8,Comércio,888983.66,17,Belo Horizonte,10,1,0
9,Indústria,1098512.64,13,Rio de Janeiro,9,3,2


### Visualizar resultados

In [62]:
# Cruzar idade e faturamento, apresentando os clusters
px.scatter(df_clientes, x='idade', y='faturamento_mensal', color='cluster')

In [63]:
# Cruzar inovação e faturamento, apresentando os clusters
px.scatter(df_clientes, x='inovacao', y='faturamento_mensal', color='cluster')

In [64]:
# Cruzar numero de funcionarios e faturamento, apresentando os clusters
px.scatter(df_clientes, x='numero_de_funcionarios', y='faturamento_mensal', color='cluster')

### Salvar o modelo e pipeline de transformação

In [65]:
import joblib

# Salvar o modelo
joblib.dump(best_kmeans, 'modelo_clusterizacao_clientes.pkl')

# Salvar o pipeline
joblib.dump(preprocessor, 'pipeline_clusterizacao_clientes.pkl')

['pipeline_clusterizacao_clientes.pkl']

### Aplicação Batch no Gradio

In [66]:
import gradio as gr

modelo = joblib.load('modelo_clusterizacao_clientes.pkl')
preprocessor = joblib.load('pipeline_clusterizacao_clientes.pkl')

def clustering(arquivo):
    # Carregar o CSV em um DataFrame
    df_empresas = pd.read_csv(arquivo.name)
    
    # Transformar os dados do Dataframe para o formado do KMeans
    X_transformed = preprocessor.fit_transform(df_empresas)
    
    # Treinar modelo
    modelo.fit(X_transformed)  
    
    # Criar a coluna cluster no DataFrame
    df_empresas['cluster'] = modelo.labels_
    df_empresas.to_csv('./clusters.csv', index=False)
    
    return './clusters.csv'

In [68]:
# Criar interface Gradio

app = gr.Interface(
    clustering,
    gr.File(file_types=['.csv']),
    "file"
)

# Rodar a aplicação
app.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
